In [15]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

import pandas as pd
import glob as glob

len_delta = 38
len_theta = 38
len_alpha = 70
len_beta = 133
len_gamma = 259

idx_delta = 4
idx_theta = 3
idx_alpha = 2
idx_beta = 1
idx_gamma = 0

In [16]:
def layer(W, num_wires):
    for i in range(num_wires):
        qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)
        
    for j in range(num_wires):
        if j != num_wires-1:
            qml.CNOT(wires=[j, j+1])
        else:
            qml.CNOT(j, 0)

In [55]:
def get_raw_angles(x, num_wires):
    angles = []
    
    squared_sum = lambda input_arr: sum([d**2 for d in input_arr])
    
    for k in range(num_wires):
        ang_k = []
        for j in range(2**(num_wires-k-1)):
            num_idx = (2*j+1)*(2**k)
            den_idx = j*(2**(k+1))
            ang = 2 * np.arcsin(np.sqrt(squared_sum(x[num_idx : num_idx+(2**k-1)+1])) / np.sqrt(squared_sum(x[den_idx : den_idx+2**(k+1)]) + 1e-12))
            ang_k.append(ang)
            
        angles.insert(0, ang_k)
            
    return angles

In [58]:
# TODO: Implement:-
# 1. Gray code
# 2. Alpha angle to theta angle tranformation (Not sure?)

def get_angles(x):
    num_wires = int(np.log(len(x)) / np.log(2))
    raw_ang = get_raw_angles(x, num_wires)
    angles = []
    print(raw_ang)
    
    angles.append(raw_ang[0][0])
    
    for i in range(num_wires-1, 0, -1):
        for a in raw_ang[i]:
            pass
            
            
    return np.array(angles)



x = np.array([0.53896774, 0.79503606, 0.27826503, 0.0])
get_angles(x)

[[0.5639746511205705], [1.9500920812754716, 0.0]]


array([0.56397465])

In [19]:
def create_ctrl_rot(num_wires):
    if num_wires == 2:
        return [num_wires-1, num_wires-1]
    else:
        ctrl_idxs = create_ctrl_rot(num_wires-1)
        ctrl_idxs[-1] += 1
        return ctrl_idxs + ctrl_idxs

In [20]:
def UCRy(angles, wires):
    num_ctrl_rot = 2 ** len(wires) - 1
    num_repetitions = len(wires)-1
    ctrl_idxs = [len(wires)-i-1 for i in create_ctrl_rot(len(wires))]
    
    i = 0
    for idx in ctrl_idxs:
        qml.RY(angles[i], wires=wires[-1])
        qml.CNOT(wires=[idx, wires[-1]])
        i += 1

In [21]:
def XUCRy(angles, wires):
    for i in wires[:-1]:
        qml.PauliX(wires=i)
        
    UCRy(angles, wires)
    
    for i in wires[:-1]:
        qml.PauliX(wires=i)

In [23]:
def statePreparation(a, num_wires):
    idx = 1
    for i in range(num_wires):
        if i == (len(a)//2)+1:
            qml.RY(a[i], wires=i)
        else:
            wires = [num for num in range(num_wires-i+1)]
            UCRy(a[idx : (idx+(2**(num_wires-i)))], wires)
            idx += (2**(num_wires-i))

    
    for i in range(num_wires-1, 0, -1):
        wires = [num for num in range(num_wires-i+1)]
        XUCRy(a[idx : (idx+(2**(num_wires-i)))], wires)
        idx += (2**(num_wires-i))

In [24]:
def normalize(list_data):
    for i in range(2):
        mu = list_data[i].mean()
        sigma = list_data[i].std()
        
        list_data[i] = (list_data[i] - mu) / sigma
        for j in range(len(list_data[i])):
            if list_data[i][j] > 2:
                list_data[i][j] = 2
            elif list_data[i][j] < -2:
                list_data[i][j] = -2
                
        list_data[i] = (list_data[i] / 5) + 0.5
            
    return list_data

In [26]:
# files = glob.glob('./DWT_TextDump/*/*.txt')
# data = pd.read_csv(files[0], delimiter='\t', header=None)

# list_data = [np.asanyarray(data[3])[:32], np.asanyarray(data[4])[:32]]

# norm_data = normalize(list_data)

# x = norm_data[0] # Load theta wave values
x = np.array([0.53896774, 0.79503606, 0.27826503, 0.0])
ang = get_angles(x)

# num_qubits = int(np.log(len(norm_data[0]))/np.log(2))
num_qubits=2

dev = qml.device('default.qubit', wires=num_qubits)

@qml.qnode(dev)
def test(angles=None):
    statePreparation(angles, num_qubits)
    return qml.expval(qml.PauliZ(0))

test(angles=ang)

print("x               : ", x)
print("angles          : ", ang)
print("amplitude vector: ", np.real(dev._state))

1 2 0 2
3 4 2 4
2 4 0 4
x               :  [0.53896774 0.79503606 0.27826503 0.        ]
angles          :  [ 0.56397465  0.         -0.          0.97504604 -0.97504604]
amplitude vector:  [5.38967743e-01 7.95036065e-01 2.78265032e-01 9.59954909e-18]
